In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.mixture import GaussianMixture

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
df = pd.read_csv('/kaggle/input/bank-customer-segmentation/bank_transactions.csv')
df.head()

In [5]:
df.info()

In [6]:
df.isna().sum()

The columns with NA values account for less than 1% of our data, so I'm just going to drop them. I don't understand why CustAccountBalance would have unknown values, the only thought I have is that they have no account, or a total balance of 0, so if I kept them, I would probaby just fill with 0 instead of a median value. CustGender could matter, that's something I'd have to explore before considering if I would want to nn impute this, and the location could matter as well, with the same deal. However, we have enough data to ignore doing this.

In [7]:
df = df.dropna()
df['CustomerDOB'].value_counts()

There are a lot of birthdates on January 1st, which seems  alittle suspicious. There's also a lot of customers birth DOB on 1/1/1800, having 70x the number as the next highest date. This is probably some default the bank has where this information is unknown, althought that is a little suspicous. I'll remove them.

In [ ]:
df = df.loc[~(df['CustomerDOB'] == '1/1/1800')]
df['CustomerDOB'].value_counts()

In [ ]:
df['CustomerDOB'] = pd.to_datetime(df['CustomerDOB'], format = '%d/%m/%y')
df.head()

In [ ]:
df.loc[df['CustomerDOB'].dt.year >= 2021, ['CustomerDOB']] -= pd.DateOffset(years = 100)
df.head()

In [ ]:
df['TransactionDate'].value_counts()

All the trnasactions took place in a roughly two month period from August to October, this could account for the low transaction frequency

In [ ]:
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], format = '%d/%m/%y')

In [ ]:
sns.histplot(x = df['TransactionDate'].dt.month, bins = 3, binwidth = 1)
plt.title('Number of transactions in each month')

In [ ]:
df['Age'] = (pd.to_datetime('today') - df['CustomerDOB'])/np.timedelta64(1, 'Y')
df['DaysSinceTransaction'] = (pd.to_datetime('today') - df['TransactionDate'])/np.timedelta64(1, 'D')
df['DaysSinceTransaction'] = df['DaysSinceTransaction'] - df['DaysSinceTransaction'].min()
temp = df[['CustomerID', 'TransactionID']].groupby(by = 'CustomerID', as_index = False, sort = False).count().reset_index()

In [ ]:
temp = temp.drop(columns = 'index')
temp.rename(columns = {'TransactionID' : 'TransactionFrequency'})
df = df.merge(right = temp, on = 'CustomerID')
df.head()

In [ ]:
df = df.rename(columns = {'TransactionID_y' : 'TransactionFrequency',
                         'DaysSinceTransaction' : 'Recency'})

In [ ]:
rmf = df.drop(columns = ['CustGender', 'CustLocation', 'CustLocation',
                         'CustAccountBalance', 'TransactionTime', 'Age']
             ).groupby(by = 'CustomerID').agg({'Recency' : 'min',
                                               'TransactionFrequency': 'first',
                                               'TransactionAmount (INR)' : 'mean'})
df = df.rename(columns = {'TransactionAmount (INR)' : 'AverageTransactionAmount'})
rmf = rmf.rename(columns = {'TransactionAmount (INR)' : 'AverageTransactionAmount'})
rmf.head()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize = (15, 5))
axes = axes.flatten()

sns.countplot(x = 'Recency', data = rmf, ax = axes[0])
sns.histplot(x = 'TransactionFrequency', data = rmf, ax = axes[1])
sns.scatterplot(x = 'AverageTransactionAmount', y = 'Recency', data = rmf, ax = axes[2])
plt.tight_layout()

In [ ]:
def recency_score(value, quartiles):
    if value < quartiles[0.25]:
        return 4
    if value < quartiles[0.5]:
        return 3
    if value < quartiles[.75]:
        return 2
    else:
        return 1

def monetary_score(value, quartiles):
    if value < quartiles[0.25]:
        return 1
    if value < quartiles[0.5]:
        return 2
    if value < quartiles[0.75]:
        return 3
    else:
        return 4
    
quartiles = rmf.quantile([0.25, 0.5, 0.75]).to_dict()

rmf['recency_score'] = rmf['Recency'].apply(recency_score, quartiles = quartiles['Recency'],)
rmf['frequency_score'] = rmf['TransactionFrequency'].astype(int)
rmf.loc[rmf['frequency_score'] > 4, 'frequency_score'] = 4
rmf['monetary_score'] = rmf['AverageTransactionAmount'].apply(monetary_score, quartiles = quartiles['AverageTransactionAmount'],)
rmf['total_score'] = rmf['recency_score'] + rmf['frequency_score'] + rmf['monetary_score']

In [ ]:
fig, axes = plt.subplots(1, 3, figsize = (20, 10))
axes = axes.flatten()

recency = rmf.groupby(by = 'TransactionFrequency').mean().reset_index()
avg_amount = rmf.groupby(by = 'TransactionFrequency').mean().reset_index()


sns.scatterplot(x = 'total_score', y = 'AverageTransactionAmount', hue = 'TransactionFrequency',
            data = avg_amount, ax = axes[0])
axes[0].set_title('''Transaction Amount vs Total Score 
                  \n Averaged over Transaction Frequency ''')
sns.scatterplot(x = 'total_score', y = 'Recency', data = recency, hue = 'TransactionFrequency',
            ax = axes[1])
axes[1].set_title('''Recency vs Total Score \n
                    Averaged over Transaction Frequency''')
sns.countplot(x = 'total_score', data = rmf, ax = axes[2])
axes[2].set_title('Number of Customers in each score range')
plt.tight_layout()

In [ ]:
rmf[rmf.total_score == 12].count()

The average transaction is mostly constant for the transaction frequency range, except for the sharp increase in the range of 4 - 5 transactions over the three months, and a sudden sharp decrease for the most frequent (and significantly rarer) 6 transactions over that interval. Since 4 - 6 transactions all give a frequency score of 4, one would expect that they share the same average total score, however this is clearly not the case, with the average total score increasing from 4-6 transactions.

When we look at the recency, we see that the average recency decreases with the average frequency (which shouldn't be shocking), which will compensate for the reduced average transactional amount.

In [ ]:
rmf.groupby(by = 'total_score').describe().T

Unsurpisingly, due to the low frequency of transactions, most people fall around a total score of the 5 - 7  out of 12. There are similar numbers of 3, 4 to 8, 9, and very few 10 and above. About 1% of the people have a score above 10. Moving on to clustering:

In [ ]:
df.head()

In [ ]:
df_cluster = df.drop(columns = ['CustomerID', 'CustomerDOB', 'TransactionDate', 'TransactionTime'])
df_cluster['CustGender'] = df_cluster['CustGender'].map(lambda x: 1 if x == 'F' else 0)
locations = df_cluster.CustLocation.unique()
locations = {place : idx for idx, place in enumerate(locations)}
df_cluster['CustLocation'] = df_cluster['CustLocation'].map(locations)
df_cluster.head()

In [ ]:
#df_cluster.rename(columns = {'TransactionAmount (INR)' : 'AverageTransactionAmount'})
numeric_to_locations = {idx : location for idx, location in enumerate(locations.keys())}
df_cluster['CustLocation'].value_counts().head(10)

There are over 1000 locations, so I believe they correspond to the bank's actual location as opposed to a providence. I'm going to just take the tenth largest location, as KMeans explodes with n

In [ ]:
location_9 = df_cluster.loc[df_cluster.CustLocation == 9]
location_9.head()

In [ ]:
del(df)
del(rmf)

I'm going to ignore global clustering, which may give insights to topics such as rural vs urban, and may give insights to demographics, ie is there a difference in banking between more conservative and more liberals, or with respect to religiousosity. The dataset is too large for batchedkmeans to find clusters in a reasonable time.

In [ ]:
ss = StandardScaler()
ct = ColumnTransformer([('Standard Scaling', ss, 
                    ['CustAccountBalance', 'AverageTransactionAmount',
                    'Age', 'Recency'])], remainder = 'passthrough')

clusters = range(2, 10)
X = location_9.drop(columns = ['TransactionID_x', 'CustLocation'])
X = ct.fit_transform(X)

In [ ]:
def plt_silhouette(clusters, clusterer):
    #This silhouette analysis is from SKLearn : https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
    for cluster in clusters:
        fig, ax = plt.subplots(1, 1, figsize = (10, 10))

        ax.set_xlim([0, 1])
        ax.set_ylim([0, len(X) + (cluster + 1) * 10])
        
        if clusterer == MiniBatchKMeans:
            md = MiniBatchKMeans(n_clusters = cluster)
            preds = md.fit_predict(X)
            md_name = 'MiniBatchKMeans'
        else:
            md = GaussianMixture(n_components = cluster)
            preds = md.fit_predict(X)
            md_name = 'GaussianMixture'
        
        sil_average = silhouette_score(X, preds)
        print(f'For {cluster} clusters,\n the average silhouette score is {sil_average}')
        sample_sil = silhouette_samples(X, preds)
        y_lower = 10

        for i in range(cluster):
            cluster_sil = sample_sil[preds == i]
            cluster_sil.sort()
            size_cluster = cluster_sil.shape[0]
            y_upper = y_lower + size_cluster
            color = cm.nipy_spectral(float(i) / cluster)
            ax.fill_betweenx(np.arange(y_lower, y_upper),
                            0,
                            cluster_sil,
                            facecolor = color,
                            edgecolor = color)
            ax.text(-0.05, y_lower + 0.5 * size_cluster, str(i))
            y_lower = y_upper + 10
            ax.axvline(x = sil_average, color = 'red', linestyle = '--')
            ax.set_yticks([])
            ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
            ax.set_title(f'Silhouette analysis using {md_name} for {cluster} clusters', fontweight = 'bold')

In [ ]:
plt_silhouette(clusters, MiniBatchKMeans)

In [ ]:
plt_silhouette(clusters, GaussianMixture)

With Silhouette score, you'd like  for the the groups to be of roughly equal sizes, and all of the clusters should have at least the average silhouette score. With regards to this, KMeans with two clusters seem to be the best, having a signifcantly higher silhouette score, and while one cluster is significantly smaller than the other, this is probably due to the large imbalance in the frequency of transactions. Let's see what happens when we exclude the frequency

In [ ]:
X = location_9.drop(columns = ['TransactionID_x', 'CustLocation', 'TransactionFrequency'])
X = ct.fit_transform(X)
plt_silhouette(clusters, MiniBatchKMeans)

In [ ]:
plt_silhouette(clusters, GaussianMixture)

Selecting Clusters is a little tough. Two clusters, if we include the transaction frequency could make sense since that should leave the clusters as people with only one transaction, and people with multiple transactions. KMF analysis showed that there is no practical difference between these groups, however (look at either recenvy vs total score. That leaves either three or four clusters while excluding the frequency. The issue with three clusters is that one of the clusters has a below average silhouette score, while with four clusters, we have one class that has significantly lower numbers. Let's see what both have to offer.

In [ ]:
km3 = MiniBatchKMeans(n_clusters = 3)
km4 = MiniBatchKMeans(n_clusters = 4)

location_9['Three_Clusters'] = km3.fit_predict(X)
location_9['Four_Clusters'] = km4.fit_predict(X)

location_9.head()



In [ ]:
#location_9['CustGender'] = location_9['CustGender'].map(lambda x: 'F' if x == 1 else 'M')
location_9['TransactionFrequency'] = location_9['TransactionFrequency'].astype(int)
fig, axes = plt.subplots(6, 2, figsize = (15, 15))
axes = axes.flatten()

continuous_columns = location_9.drop(columns = ['TransactionID_x','CustLocation', 'Three_Clusters',
                                                'Four_Clusters', 'CustGender', 'TransactionFrequency']).columns

cat_columns = location_9[['CustGender', 'TransactionFrequency']]

for idx, c_name in enumerate(cat_columns):
    sns.countplot(x = c_name, hue = 'Three_Clusters',  data = location_9, ax = axes[2 * idx])
    sns.countplot(x = c_name, hue = 'Four_Clusters', data = location_9, ax = axes[2 * idx + 1])
    axes[2 * idx].set_title(f'{c_name} Three Clusters')
    axes[2 * idx + 1].set_title(f'{c_name} Four Clusters')

for idx, c_name in enumerate(continuous_columns):
    sns.scatterplot(y = 'Three_Clusters', x = c_name,  data = location_9, ax = axes[2 * (idx + 2)])
    sns.scatterplot(y = 'Four_Clusters', x = c_name, data = location_9, ax = axes[2 * (idx + 2) + 1])
    axes[2 * idx].set_title(f'{c_name} Three Clusters')
    axes[2 * idx + 1].set_title(f'{c_name} Four Clusters')
    
plt.tight_layout()

There's nothing really indicating what's different about the groups. For the four clusters, the gender ratio, and transaction frequency ratios look roughly the same across the clusters, ie the ratio of M : F in cluster 3 appears to be the same ratio as cluster 0. Cluster 0, 1, 3 have the same account balance, and average transaction amount, while cluster 2 has some values exceeding these values. Age is really the same, adn the only other major difference is the recency (DaysSinceTransaction), where cluster 3 has mostly low recencies (as in days, not score), cluster 0 has middling, cluster 1 has high recency, however cluster 2 is all over the place.

A somewhat similar story exists for the three cluster situation.